In [1]:
import os
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
from keras.utils import to_categorical
import numpy as np
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras import backend as K
from matplotlib.pyplot import *
import matplotlib

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'C:/Users/hp/Downloads/data/model/top_model_vgg/bottleneck_fc_model.h5'
train_data_dir = 'C:/Users/hp/Downloads/data/train/'
validation_data_dir = 'C:/Users/hp/Downloads/data/validation'
nb_train_samples = 1200
nb_validation_samples = 198
epochs = 50
batch_size = 16

In [6]:
#train_data = np.load(open('C:/Users/hp/Downloads/data/model/bottle_neck_features/bottleneck_features_train.npy', 'rb'))

In [7]:
#vgg_16_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [8]:
def create_model():
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [9]:
def load_trained_model(weights_path):
    model = create_model()
    model.load_weights(weights_path)
    
    return model

In [10]:
def generateBottleneckfeatures(model, generator):
    bottleneck_features = model.predict_generator(generator, nb_train_samples // batch_size)
    #np.save(open('D:/retinal_data_set_visioncare/visualization_features/vgg16/bottle_neck_features/bottleneck_features_train_block5.npy', 'wb'), bottleneck_features_train)
    nsamples, nx, ny, nd = bottleneck_features.shape
    reshaped_bottleneck_features = bottleneck_features.reshape((nsamples,nx*ny*nd))
    
    return reshaped_bottleneck_features

In [3]:
# Returns a compiled model identical to the previous one
model = load_model('C:/Users/hp/Downloads/data/model/top_model_vgg/final_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [4]:
# Get all the classes from the original train directory
classes = os.listdir(train_data_dir)

# keep features with its corrosponding file names as dictionary
features = {}


In [5]:
classes

['cats', 'dogs']

In [6]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer("desnse_256").output)
intermediate_layer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [ ]:
feature_list = []

for idx, level in enumerate(classes):
    retina_root = train_data_dir + level + '/'
    retina_root_list = os.listdir(retina_root)
    
    for index, relative_path in enumerate(retina_root_list):
        source = retina_root + relative_path
        img = image.load_img(source, target_size=(img_width, img_height))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # get the features 
        extract_features = intermediate_layer_model.predict(x)
        #features[source] = extract_features
        feature_list.append({'path':source, 'feature':extract_features})